In [1]:
import cv2
import mediapipe as mp
import os
import pandas as pd
import numpy as np

# Path to training data
trainPath = os.path.join(os.getcwd(), 'ASL_Dataset', 'Train')

# This list will hold one row per hand detection:
# Each row contains 21 landmarks * 3 coordinates = 63 values, plus one label column.
data = []

# Initialize MediaPipe Hands module
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.4)

# Loop over each folder (assumed to be the label)
for folder in os.listdir(trainPath):
    if folder.lower() == 'nothing':
        continue
    folderPath = os.path.join(trainPath, folder)
    success = 0
    total = 0
    print(f"Processing folder: {folder}")
    
    # Loop over each image file in the folder
    for file in os.listdir(folderPath):
        file_path = os.path.join(folderPath, file)
        image = cv2.imread(file_path)
        total += 1
        # Skip images that failed to load
        if image is None:
            continue
        
        # ===================== Preprocessing Steps =====================
        # 1. Resize the image to a standard size (e.g., 256x256)
        # image = cv2.resize(image, (400,400))
        
        # 2. Apply Gaussian blur to reduce noise
        image = cv2.GaussianBlur(image, (5, 5), 0)
        # ================================================================
        
        # Convert the preprocessed image from BGR to RGB for MediaPipe processing
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        results = hands.process(rgb_image)

        # Check if any hand landmarks were detected
        if results.multi_hand_landmarks:
            # Process each detected hand in the image
            for hand_landmarks in results.multi_hand_landmarks:
                row = []  # This row will hold all the landmark coordinates for one hand
                # For each of the 21 landmarks, append x, y, and z coordinates
                for landmark in hand_landmarks.landmark:
                    row.extend([landmark.x, landmark.y, landmark.z])
                # Append the label (folder name) as the last entry in the row
                row.append(folder)
                # Add the row to our data list
                data.append(row)
            success += 1
    print(f"Folder: {folder}, Count: {success}/{total}")

# Create column names: x0, y0, z0, ..., x20, y20, z20, and a 'label' column.
columns = []
num_landmarks = 21  # MediaPipe Hands always returns 21 landmarks per detected hand.
for i in range(num_landmarks):
    columns.extend([f'x{i}', f'y{i}', f'z{i}'])
columns.append('label')

# Create the DataFrame
df = pd.DataFrame(data, columns=columns)

I0000 00:00:1742475059.798606 23409357 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1742475059.805853 23409866 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742475059.810191 23409867 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742475059.826822 23409863 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Processing folder: R
Folder: R, Count: 5966/5966
Processing folder: U
Folder: U, Count: 4542/4542
Processing folder: I
Folder: I, Count: 5996/5996
Processing folder: N
Folder: N, Count: 4886/5996
Processing folder: G
Folder: G, Count: 5996/5996
Processing folder: Z
Folder: Z, Count: 5803/5996
Processing folder: T
Folder: T, Count: 5648/5648
Processing folder: S
Folder: S, Count: 5996/5996
Processing folder: A
Folder: A, Count: 5996/5996
Processing folder: F
Folder: F, Count: 5996/5996
Processing folder: O
Folder: O, Count: 5925/5996
Processing folder: H
Folder: H, Count: 5996/5996
Processing folder: Space
Folder: Space, Count: 5504/5886
Processing folder: M
Folder: M, Count: 5840/5996
Processing folder: J
Folder: J, Count: 5996/5996
Processing folder: C
Folder: C, Count: 5996/5996
Processing folder: D
Folder: D, Count: 5760/5996
Processing folder: V
Folder: V, Count: 5995/5996
Processing folder: Q
Folder: Q, Count: 5795/5996
Processing folder: X
Folder: X, Count: 5996/5996
Processing f

In [2]:
df.info()
df['label'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156270 entries, 0 to 156269
Data columns (total 64 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   x0      156270 non-null  float64
 1   y0      156270 non-null  float64
 2   z0      156270 non-null  float64
 3   x1      156270 non-null  float64
 4   y1      156270 non-null  float64
 5   z1      156270 non-null  float64
 6   x2      156270 non-null  float64
 7   y2      156270 non-null  float64
 8   z2      156270 non-null  float64
 9   x3      156270 non-null  float64
 10  y3      156270 non-null  float64
 11  z3      156270 non-null  float64
 12  x4      156270 non-null  float64
 13  y4      156270 non-null  float64
 14  z4      156270 non-null  float64
 15  x5      156270 non-null  float64
 16  y5      156270 non-null  float64
 17  z5      156270 non-null  float64
 18  x6      156270 non-null  float64
 19  y6      156270 non-null  float64
 20  z6      156270 non-null  float64
 21  x7      15

label
W        5996
C        5996
I        5996
L        5996
G        5996
K        5996
B        5996
S        5996
A        5996
F        5996
E        5996
H        5996
X        5996
J        5996
V        5995
R        5966
O        5925
M        5840
Z        5803
Q        5795
D        5760
Y        5720
T        5648
Space    5504
P        4942
N        4886
U        4542
Name: count, dtype: int64

In [ ]:
df.to_csv('dataset1GaussianBlurMin4.csv', index=False)

In [ ]:
import pandas as pd
df = pd.read_csv('dataset1GaussianBlurMin4.csv')
df.info()
df['label'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156270 entries, 0 to 156269
Data columns (total 64 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   x0      156270 non-null  float64
 1   y0      156270 non-null  float64
 2   z0      156270 non-null  float64
 3   x1      156270 non-null  float64
 4   y1      156270 non-null  float64
 5   z1      156270 non-null  float64
 6   x2      156270 non-null  float64
 7   y2      156270 non-null  float64
 8   z2      156270 non-null  float64
 9   x3      156270 non-null  float64
 10  y3      156270 non-null  float64
 11  z3      156270 non-null  float64
 12  x4      156270 non-null  float64
 13  y4      156270 non-null  float64
 14  z4      156270 non-null  float64
 15  x5      156270 non-null  float64
 16  y5      156270 non-null  float64
 17  z5      156270 non-null  float64
 18  x6      156270 non-null  float64
 19  y6      156270 non-null  float64
 20  z6      156270 non-null  float64
 21  x7      15

label
W        5996
C        5996
I        5996
L        5996
G        5996
K        5996
B        5996
S        5996
A        5996
F        5996
E        5996
H        5996
X        5996
J        5996
V        5995
R        5966
O        5925
M        5840
Z        5803
Q        5795
D        5760
Y        5720
T        5648
Space    5504
P        4942
N        4886
U        4542
Name: count, dtype: int64